In [ ]:
import pandas as pd 
import numpy as np
import plotly.express as px
import seaborn as sns

df = pd.read_csv(r"C:\Users\archi\Downloads\ListofCountriesbySugarcaneProduction.csv")

In [ ]:
df.head(15)

In [ ]:
df.shape

In [ ]:
df.info()

# Data Cleaning

format text is left , numeric is right align , col name is centre , caps
Convert object type to integer/float type
remove . or , point  properly in numeric
add , correctly in production per kg 
correct the value for yield like for 8	Guatemala	North America	33.533.403	1.938,114	259.85	129.049,3, yield is wrong by decimal points so correct it.
73	Somalia	Africa	210,62	13873	5,731	36.7492   here it should be 21,062 and yield value points  so check correctly where . or , comes
check for -ve / +ve
Check for dupicates
Check for null values.
check for outlier for int / float values
Check for country and continent format start with capital and then small.
Check range of countries and continent of it are correct or not. 
check if data is sorted correctly


for data visualization
top 10 countries after grpby by countries.
group by continent, then top 5 continent by source

which country, has most production by person value

In [ ]:
df.drop('Index', axis=1, inplace=True)

In [ ]:
df.rename(columns=lambda x: x.upper(), inplace=True)

In [ ]:
df.style.set_properties(subset=['COUNTRY' , 'CONTINENT'], text_align='left')

In [ ]:
df.isna().sum()

In [ ]:
duplicate_rows = df[df.duplicated()]
print(duplicate_rows)

In [ ]:
rows_with_null_acreage = df[df['ACREAGE (HECTARE)'].isnull()]
rows_with_null_acreage

In [ ]:
additional_data = df[df['COUNTRY'] == 'Djibouti']
additional_data

Djibouti country has only 1 data pressent, so instead of dropping this , convert NaN to 0.0 object type . Convert to int and float after dropping it and then find averge of acreage based on other countries of Africa.

In [ ]:
df.loc[df['COUNTRY'] == 'Djibouti', 'ACREAGE (HECTARE)'].fillna('0', inplace=True)

In [ ]:
df.loc[df['COUNTRY'] == 'Djibouti', 'YIELD (KG / HECTARE)'].fillna('0', inplace=True)

In [ ]:
df[df['COUNTRY'] == 'Djibouti']

In [ ]:
df['PRODUCTION (TONS)'] = df['PRODUCTION (TONS)'].str.replace('.', '')
df['PRODUCTION (TONS)'] = df['PRODUCTION (TONS)'].astype(str).astype(int)

In [ ]:
df

36,Zimbabwe,Africa,3.483.000,234563,43.5,80.069  and many more acreage is before 1 digit then it should be 43500 and not 435 , so first check after decimal how many digit if 3 simply convert other wise complete total 3 digit after decimal by adding 0.

In [ ]:
df['ACREAGE (HECTARE)'].fillna('0', inplace=True)

In [ ]:
def convert_acreage(acreage):
    acreage_str = str(acreage)
    parts = acreage_str.split('.')
    if len(parts) > 1:
        last_decimal_part = parts[-1]
        if len(last_decimal_part) < 3:  
            last_decimal_part += '0' * (3 - len(last_decimal_part))  
            parts[-1] = last_decimal_part
            return ''.join(parts)
        else :
            return ''.join(parts)
    else:
        return acreage_str

df['ACREAGE (HECTARE)'] = df['ACREAGE (HECTARE)'].apply(convert_acreage)


In [ ]:
df['ACREAGE (HECTARE)'].dtype

In [ ]:
df[df['COUNTRY']=='Zimbabwe']

In [ ]:
df['ACREAGE (HECTARE)'] = df['ACREAGE (HECTARE)'].astype(str).astype(int)

In [ ]:
df['ACREAGE (HECTARE)'].dtype

In [ ]:
df['YIELD (KG / HECTARE)'] = df['YIELD (KG / HECTARE)'].str.replace('.', '')
df['YIELD (KG / HECTARE)'] = df['YIELD (KG / HECTARE)'].str.replace(',', '.')

In [ ]:
df

In [ ]:
df['YIELD (KG / HECTARE)'].dtype

In [ ]:
df['PRODUCTION PER PERSON (KG)'] = df['PRODUCTION PER PERSON (KG)'].str.replace('.', '')
df['PRODUCTION PER PERSON (KG)'] = df['PRODUCTION PER PERSON (KG)'].astype(float)

In [ ]:
df['PRODUCTION PER PERSON (KG)'].dtype

In [ ]:
mean_acreage_africa = df[df['CONTINENT'] == 'Africa']['ACREAGE (HECTARE)'].mean()
print("Mean acreage in Africa:", mean_acreage_africa)

In [ ]:
mode_acreage_africa = df[df['CONTINENT'] == 'Africa']['ACREAGE (HECTARE)'].mode().iloc[0]
print("Mode acreage in Africa:", mode_acreage_africa)

In [ ]:
mean_acreage_africa = int(mean_acreage_africa)

In [ ]:
df.loc[df['COUNTRY'] == 'Djibouti', 'ACREAGE (HECTARE)'] = mean_acreage_africa
df.loc[df['COUNTRY'] == 'Djibouti', 'YIELD (KG / HECTARE)'] = int(df.loc[df['COUNTRY'] == 'Djibouti', 'PRODUCTION (TONS)'] / (mean_acreage_africa / 1000))

In [ ]:
df.describe()

Check whether continents and countries mentioned exists?

In [ ]:
df['CONTINENT'].unique()

In [ ]:
df['COUNTRY'].unique()

In [ ]:
df['COUNTRY'].nunique()

find data where any of int or float is 0

In [ ]:
numeric_columns = df.select_dtypes(include=['int', 'float']).columns
rows_with_zero_values = df[(df[numeric_columns] == 0).any(axis=1)]
rows_with_zero_values

Population not present so not able to find production per person.

In [ ]:
df.loc[df['COUNTRY'] == 'Syria', 'ACREAGE (HECTARE)'] = df.loc[df['COUNTRY'] == 'Syria','PRODUCTION (TONS)'] / df.loc[df['COUNTRY'] == 'Syria','YIELD (KG / HECTARE)']

# Check calculation for all yield = production in kg / acreage in hectare

In [ ]:
df['Calculated Yield (KG / HECTARE)'] = df['PRODUCTION (TONS)'] / (df['ACREAGE (HECTARE)'] / 1000)
 
length = len(str(df['YIELD (KG / HECTARE)'][0]))  
for i in range(len(df)):
    df.loc[i, 'Yield Match'] = str(df['Calculated Yield (KG / HECTARE)'][i])[:length] == str(df['YIELD (KG / HECTARE)'][i])[:length]

mismatched_df = df[df['Yield Match'] == False]

print("Rows with mismatched yield:")
mismatched_df


In [ ]:
mismatched_df.count()

51 countries mismatches with digits after decimal points , they are round off to 2 decimal places some to 3 decimal places, so correct all mismatched values and make it to 2 decimal places for all.

In [ ]:
for i in mismatched_df.index:
    df.loc[i, 'Calculated Yield (KG / HECTARE)'] = df.loc[i, 'PRODUCTION (TONS)'] / (df.loc[i, 'ACREAGE (HECTARE)'] / 1000)
    df.loc[i, 'YIELD (KG / HECTARE)'] = round(df.loc[i, 'Calculated Yield (KG / HECTARE)'], 2)


In [ ]:
df.drop(columns=['Calculated Yield (KG / HECTARE)' , 'Yield Match'], inplace=True)

# Find population for all except Syria

In [ ]:
df_filtered = df[df['COUNTRY'] != 'Syria'].copy()

df['POPULATION'] = df_filtered['PRODUCTION (TONS)'] * 1000 / df_filtered['PRODUCTION PER PERSON (KG)']
df.loc[df['COUNTRY'] == 'Syria', 'POPULATION'] = 0

In [ ]:
df['POPULATION'].describe()

# Checking outliers

In [ ]:
check_outliers = df.select_dtypes(include=['int', 'float']).columns

In [ ]:
for column in check_outliers:
    fig = px.box(df, y=column, title=f'Box Plot of {column}', labels={'value': 'Value'})
    fig.update_layout(showlegend=False)  # Hide legend
    fig.show()

# Which continent produces maximum production?
# Do number of countries in a continent affect production of sugarcane?

In [ ]:
continent_counts = df['CONTINENT'].value_counts()
fig = px.bar(x=continent_counts.index, y=continent_counts.values)
fig.update_layout(title='Value Counts of CONTINENT', xaxis_title='CONTINENT', yaxis_title='Count')
fig.show()

In [ ]:
continent_production = df.groupby('CONTINENT')['PRODUCTION (TONS)'].sum().reset_index(name='Total Production')
continent_production = continent_production.sort_values(by='Total Production', ascending=False)

fig = px.bar(continent_production, x='CONTINENT', y='Total Production')
fig.update_layout(title='Production by Continent', xaxis_title='Continent', yaxis_title='Total Production')
fig.show()

It does not affect , Africa having more countries but less production than South America.

In [ ]:
fig = px.pie(continent_production, values='Total Production', names='CONTINENT', title='Production by Continent')
fig.show()

# Distribution of production among top 5 countries

In [ ]:
df_new = df[['COUNTRY' , 'PRODUCTION (TONS)']].set_index('COUNTRY')

In [ ]:
df_new['PRODUCTION (TONS) PERCENT'] = (df_new['PRODUCTION (TONS)'] * 100) / df_new['PRODUCTION (TONS)'].sum()
top_5_countries = df_new.nlargest(5, 'PRODUCTION (TONS) PERCENT')

fig = px.pie(top_5_countries, values='PRODUCTION (TONS) PERCENT',names= top_5_countries.index,  title='Top 5 Countries by Production Percentage')
fig.show()

# Which country has more area of land ?

In [ ]:
area = df[['COUNTRY', 'ACREAGE (HECTARE)']].set_index('COUNTRY')

In [ ]:
area['PRODUCTION (TONS) PERCENT'] = (area['ACREAGE (HECTARE)'] * 100) / area['ACREAGE (HECTARE)'].sum()
top_5_countries = area.nlargest(5, 'ACREAGE (HECTARE)')

fig = px.pie(top_5_countries, values='ACREAGE (HECTARE)',names= top_5_countries.index,  title='Top 5 Countries by Area')
fig.show()

# Which country has more population ?

In [ ]:
df

In [ ]:
popul = df[['COUNTRY', 'POPULATION']].set_index('COUNTRY')

In [ ]:
top_5_countries = popul.nlargest(5, 'POPULATION')

fig = px.pie(top_5_countries, values='POPULATION',names= top_5_countries.index,  title='Top 5 Countries by Population')
fig.show()

# Which county has highest yield ?

In [ ]:
top_10_countries_by_yield = df.nlargest(10, 'YIELD (KG / HECTARE)')
fig = px.bar(data_frame= top_10_countries_by_yield, x= 'COUNTRY', y= 'YIELD (KG / HECTARE)')
fig.update_layout(title='Value Counts of Yield', xaxis_title='COUNTRY', yaxis_title='Count')
fig.show()

# Correlation

In [ ]:
df.select_dtypes(exclude=['object']).corr()

In [ ]:
sns.heatmap(df.select_dtypes(exclude=['object']).corr() , annot = True , cmap="Greens")

# Do countries with highest land produce more sugarcane ?

In [ ]:
fig = px.scatter(df, x='ACREAGE (HECTARE)', y='PRODUCTION (TONS)', hover_data=['COUNTRY'], title='Production vs Acreage')

fig.show()

# Do countries with more yield more sugarcane per hectare produces more sugarcane  in total?

In [ ]:
fig = px.scatter(df, x='YIELD (KG / HECTARE)', y='PRODUCTION (TONS)', hover_data=['COUNTRY'], title='Production vs Yield')

fig.show()